# Unit commitment problem I - Pumped storage

Inspired by example from book Einführung in Optimirungsmodelle from Nathan Sudermann-Merx page 120

Problem statement:

> Every hour, a [pumped hydro plant](https://en.wikipedia.org/wiki/Pumped-storage_hydroelectricity) can choose to _either_ pump water uphill or generate power from its turbine by letting water run downhill. At the start of the year, the reservoir contains 300 MWh of energy. At the end of the year, it must return to this amount (such that the model is more realistic of the constraints of multi-year operation). The reservoir can never go below 100 MWh and its maximal energy storage capacity is 630 MWh. When the pump is on, it draws 70 MW of power and has a 75% efficiency. The turbine can output anywhere between 0 and 90 MW of power (when the pump is not in use). Given the cost of electricity at every hour, find the pump and turbine "schedule" that would maximize operational profits (turbine energy sold - pump energy bought).

The input data is using real electricity prices from the German and Luxembourg market stored in `elspot-prices_2021_hourly_eur.csv`.



In [1]:
import polars as pl
import pyoframe as pf
from pathlib import Path

Read hourly prices from CSV file and return a DataFrame.
Special attention is paid to properly parse daylight saving time (DST) changes.

In [2]:
def read_hourly_prices():
    df = pl.read_csv(
        Path("./elspot-prices_2021_hourly_eur.csv"),
        try_parse_dates=True,
    ).drop_nulls(subset=["DE-LU"])

    df = df.select(
        pl.datetime(
            pl.col("Date").dt.year(),
            pl.col("Date").dt.month(),
            pl.col("Date").dt.day(),
            pl.col("Hours").str.slice(0, 2).cast(pl.Int32),
            time_zone="Europe/Prague",
            ambiguous=pl.when(
                pl.concat_str(pl.col("Date"), pl.col("Hours")).is_first_distinct()
            )
            .then(pl.lit("earliest"))
            .otherwise(pl.lit("latest")),
        ).alias("tick"),
        pl.col("DE-LU").str.replace(",", ".", literal=True).cast(float).alias("price"),
    )

    return df

Read input data and set parameters

In [3]:
hourly_prices = read_hourly_prices()
pump_max, turb_max = 70, 90
storage_min, storage_max = 100, 630
storage_level_init_and_final = 300
efficiency = 0.75

In [4]:
hourly_prices

tick,price
"datetime[μs, Europe/Prague]",f64
2021-01-01 00:00:00 CET,50.87
2021-01-01 01:00:00 CET,48.19
2021-01-01 02:00:00 CET,44.68
2021-01-01 03:00:00 CET,42.92
2021-01-01 04:00:00 CET,40.39
…,…
2021-12-31 19:00:00 CET,32.49
2021-12-31 20:00:00 CET,0.18
2021-12-31 21:00:00 CET,0.08


Decision variables:
1. `Pump`: binary variable indicating whether the pump is on (1) or off (0) at a given hour
2. `Turb`: continuous variable indicating the amount of energy produced by the turbine at a given hour (0 to turb_max)
3. `Storage_level`: continuous variable indicating the amount of energy stored in the reservoir at a given hour (between storage_min and storage_max)

In [5]:
m = pf.Model("unit commitment problem", solver='highs', use_var_names=True)

m.Pump = pf.Variable(hourly_prices[["tick"]], vtype=pf.VType.BINARY)
# ub is redundant since it will be set also in logical condition that pump and turbine cannot work at the same time
m.Turb = pf.Variable(hourly_prices[["tick"]], lb=0, ub=turb_max)
m.Storage_level = pf.Variable(
    hourly_prices[["tick"]], lb=storage_min, ub=storage_max
)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


In [6]:
m

<Model name=unit commitment problem vars=3 constrs=0 objective=False>

In [7]:
m.Pump

<Variable name=Pump size=8760 dimensions={'tick': 8760}>
[2021-01-01_00:00:00.000000+01:00]: Pump[2021-01-01_00:00:00.000000+01:00]
[2021-01-01_01:00:00.000000+01:00]: Pump[2021-01-01_01:00:00.000000+01:00]
[2021-01-01_02:00:00.000000+01:00]: Pump[2021-01-01_02:00:00.000000+01:00]
[2021-01-01_03:00:00.000000+01:00]: Pump[2021-01-01_03:00:00.000000+01:00]
[2021-01-01_04:00:00.000000+01:00]: Pump[2021-01-01_04:00:00.000000+01:00]
[2021-01-01_05:00:00.000000+01:00]: Pump[2021-01-01_05:00:00.000000+01:00]
[2021-01-01_06:00:00.000000+01:00]: Pump[2021-01-01_06:00:00.000000+01:00]
[2021-01-01_07:00:00.000000+01:00]: Pump[2021-01-01_07:00:00.000000+01:00]
[2021-01-01_08:00:00.000000+01:00]: Pump[2021-01-01_08:00:00.000000+01:00]
[2021-01-01_09:00:00.000000+01:00]: Pump[2021-01-01_09:00:00.000000+01:00]
[2021-01-01_10:00:00.000000+01:00]: Pump[2021-01-01_10:00:00.000000+01:00]
[2021-01-01_11:00:00.000000+01:00]: Pump[2021-01-01_11:00:00.000000+01:00]
[2021-01-01_12:00:00.000000+01:00]: Pump[20

In [8]:
m.Turb

<Variable name=Turb lb=0 ub=90 size=8760 dimensions={'tick': 8760}>
[2021-01-01_00:00:00.000000+01:00]: Turb[2021-01-01_00:00:00.000000+01:00]
[2021-01-01_01:00:00.000000+01:00]: Turb[2021-01-01_01:00:00.000000+01:00]
[2021-01-01_02:00:00.000000+01:00]: Turb[2021-01-01_02:00:00.000000+01:00]
[2021-01-01_03:00:00.000000+01:00]: Turb[2021-01-01_03:00:00.000000+01:00]
[2021-01-01_04:00:00.000000+01:00]: Turb[2021-01-01_04:00:00.000000+01:00]
[2021-01-01_05:00:00.000000+01:00]: Turb[2021-01-01_05:00:00.000000+01:00]
[2021-01-01_06:00:00.000000+01:00]: Turb[2021-01-01_06:00:00.000000+01:00]
[2021-01-01_07:00:00.000000+01:00]: Turb[2021-01-01_07:00:00.000000+01:00]
[2021-01-01_08:00:00.000000+01:00]: Turb[2021-01-01_08:00:00.000000+01:00]
[2021-01-01_09:00:00.000000+01:00]: Turb[2021-01-01_09:00:00.000000+01:00]
[2021-01-01_10:00:00.000000+01:00]: Turb[2021-01-01_10:00:00.000000+01:00]
[2021-01-01_11:00:00.000000+01:00]: Turb[2021-01-01_11:00:00.000000+01:00]
[2021-01-01_12:00:00.000000+01:0

In [9]:
m.Storage_level

<Variable name=Storage_level lb=100 ub=630 size=8760 dimensions={'tick': 8760}>
[2021-01-01_00:00:00.000000+01:00]: Storage_level[2021-01-01_00:00:00.000000+01:00]
[2021-01-01_01:00:00.000000+01:00]: Storage_level[2021-01-01_01:00:00.000000+01:00]
[2021-01-01_02:00:00.000000+01:00]: Storage_level[2021-01-01_02:00:00.000000+01:00]
[2021-01-01_03:00:00.000000+01:00]: Storage_level[2021-01-01_03:00:00.000000+01:00]
[2021-01-01_04:00:00.000000+01:00]: Storage_level[2021-01-01_04:00:00.000000+01:00]
[2021-01-01_05:00:00.000000+01:00]: Storage_level[2021-01-01_05:00:00.000000+01:00]
[2021-01-01_06:00:00.000000+01:00]: Storage_level[2021-01-01_06:00:00.000000+01:00]
[2021-01-01_07:00:00.000000+01:00]: Storage_level[2021-01-01_07:00:00.000000+01:00]
[2021-01-01_08:00:00.000000+01:00]: Storage_level[2021-01-01_08:00:00.000000+01:00]
[2021-01-01_09:00:00.000000+01:00]: Storage_level[2021-01-01_09:00:00.000000+01:00]
[2021-01-01_10:00:00.000000+01:00]: Storage_level[2021-01-01_10:00:00.000000+01:

Add constraints to the model:
1. The initial and final storage level must be equal to `storage_level_init_and_final`.
2. The intermediate storage level for next hour is calculated based on the pump and turbine operations, taking into account the efficiency of the pump and state of storage level in the previous hour.
3. The pump and turbine cannot operate simultaneously, meaning if the pump is on, the turbine must be off (and vice versa). BigM trick is used to enforce this condition.

In [10]:
m.initial_storage_level = (
        m.Storage_level.filter(
            pl.col("tick") == hourly_prices["tick"].min()
        )
        == storage_level_init_and_final
)

m.intermediate_storage_level = (
        m.Storage_level.next(dim="tick", wrap_around=True)
        == m.Storage_level + m.Pump * pump_max * efficiency - m.Turb
)

m.pump_and_turbine_xor = m.Turb <= (1 - m.Pump) * turb_max

In [11]:
m

<Model name=unit commitment problem vars=3 constrs=3 objective=False>

In [12]:
m.initial_storage_level

<Constraint name=initial_storage_level sense='=' size=1 dimensions={'tick': 1} terms=2>
[2021-01-01_00:00:00.000000+01:00]: Storage_level[2021-01-01_00:00:00.000000+01:00] = 300

In [13]:
m.intermediate_storage_level

<Constraint name=intermediate_storage_level sense='=' size=8760 dimensions={'tick': 8760} terms=35040>
[2021-01-01_00:00:00.000000+01:00]: Storage_level[2021-01-01_01:00:00.000000+01:00] - Storage_level[2021-01-01_00:00... = 0
[2021-01-01_01:00:00.000000+01:00]: Storage_level[2021-01-01_02:00:00.000000+01:00] - Storage_level[2021-01-01_01:00... = 0
[2021-01-01_02:00:00.000000+01:00]: Storage_level[2021-01-01_03:00:00.000000+01:00] - Storage_level[2021-01-01_02:00... = 0
[2021-01-01_03:00:00.000000+01:00]: Storage_level[2021-01-01_04:00:00.000000+01:00] - Storage_level[2021-01-01_03:00... = 0
[2021-01-01_04:00:00.000000+01:00]: Storage_level[2021-01-01_05:00:00.000000+01:00] - Storage_level[2021-01-01_04:00... = 0
[2021-01-01_05:00:00.000000+01:00]: Storage_level[2021-01-01_06:00:00.000000+01:00] - Storage_level[2021-01-01_05:00... = 0
[2021-01-01_06:00:00.000000+01:00]: Storage_level[2021-01-01_07:00:00.000000+01:00] - Storage_level[2021-01-01_06:00... = 0
[2021-01-01_07:00:00.000000+0

In [14]:
m.pump_and_turbine_xor

<Constraint name=pump_and_turbine_xor sense='<=' size=8760 dimensions={'tick': 8760} terms=26280>
[2021-01-01_00:00:00.000000+01:00]: Turb[2021-01-01_00:00:00.000000+01:00] +90 Pump[2021-01-01_00:00:00.000000+01:00... <= 90
[2021-01-01_01:00:00.000000+01:00]: Turb[2021-01-01_01:00:00.000000+01:00] +90 Pump[2021-01-01_01:00:00.000000+01:00... <= 90
[2021-01-01_02:00:00.000000+01:00]: Turb[2021-01-01_02:00:00.000000+01:00] +90 Pump[2021-01-01_02:00:00.000000+01:00... <= 90
[2021-01-01_03:00:00.000000+01:00]: Turb[2021-01-01_03:00:00.000000+01:00] +90 Pump[2021-01-01_03:00:00.000000+01:00... <= 90
[2021-01-01_04:00:00.000000+01:00]: Turb[2021-01-01_04:00:00.000000+01:00] +90 Pump[2021-01-01_04:00:00.000000+01:00... <= 90
[2021-01-01_05:00:00.000000+01:00]: Turb[2021-01-01_05:00:00.000000+01:00] +90 Pump[2021-01-01_05:00:00.000000+01:00... <= 90
[2021-01-01_06:00:00.000000+01:00]: Turb[2021-01-01_06:00:00.000000+01:00] +90 Pump[2021-01-01_06:00:00.000000+01:00... <= 90
[2021-01-01_07:00:00

Set the objective function to maximize the profit from selling energy produced by the turbine minus the cost of energy bought for pumping

In [15]:
hourly_profit = (m.Turb - pump_max * m.Pump) * hourly_prices
m.maximize = pf.sum(hourly_profit)

Hessian has dimension 26281 but no nonzeros, so is ignored


In [16]:
m

<Model name=unit commitment problem vars=3 constrs=3 objective=True>

In [17]:
m.objective

<Objective size=1 dimensions={} terms=17506>
objective: 50.87 Turb[2021-01-01_00:00:00.000000+01:00] -3560.9 Pump[2021-01-01_00:00:00.00...

Now that trinity of model, variables and constraints is set, we can optimize the model.

In [18]:
m.attr.RelativeGap = 1e-5

m.optimize()

MIP  has 17521 rows; 26281 cols; 61322 nonzeros; 8760 integer variables (8760 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e-02, 4e+04]
  Bound  [1e+00, 6e+02]
  RHS    [0e+00, 0e+00]
Presolving model
17520 rows, 26279 cols, 52558 nonzeros  0s
17518 rows, 26277 cols, 52554 nonzeros  0s

Solving MIP model with:
   17518 rows
   26277 cols (8760 binary, 0 integer, 0 implied int., 17517 continuous, 0 domain fixed)
   52554 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol       

In [19]:
m.Storage_level.solution

tick,solution
"datetime[μs, Europe/Prague]",f64
2021-01-01 00:00:00 CET,300.0
2021-01-01 01:00:00 CET,300.0
2021-01-01 02:00:00 CET,300.0
2021-01-01 03:00:00 CET,300.0
2021-01-01 04:00:00 CET,300.0
…,…
2021-12-31 19:00:00 CET,232.5
2021-12-31 20:00:00 CET,142.5
2021-12-31 21:00:00 CET,195.0


In [20]:
m.Turb.solution

tick,solution
"datetime[μs, Europe/Prague]",f64
2021-01-01 00:00:00 CET,0.0
2021-01-01 01:00:00 CET,0.0
2021-01-01 02:00:00 CET,0.0
2021-01-01 03:00:00 CET,0.0
2021-01-01 04:00:00 CET,0.0
…,…
2021-12-31 19:00:00 CET,90.0
2021-12-31 20:00:00 CET,0.0
2021-12-31 21:00:00 CET,0.0


In [21]:
m.Pump.solution

tick,solution
"datetime[μs, Europe/Prague]",i64
2021-01-01 00:00:00 CET,0
2021-01-01 01:00:00 CET,0
2021-01-01 02:00:00 CET,0
2021-01-01 03:00:00 CET,0
2021-01-01 04:00:00 CET,1
…,…
2021-12-31 19:00:00 CET,0
2021-12-31 20:00:00 CET,1
2021-12-31 21:00:00 CET,1


In [22]:
m.objective.value

6341867.15

In [23]:
hourly_profit.evaluate()

tick,solution
"datetime[μs, Europe/Prague]",f64
2021-01-01 00:00:00 CET,0.0
2021-01-01 01:00:00 CET,0.0
2021-01-01 02:00:00 CET,0.0
2021-01-01 03:00:00 CET,0.0
2021-01-01 04:00:00 CET,-2827.3
…,…
2021-12-31 19:00:00 CET,2924.1
2021-12-31 20:00:00 CET,-12.6
2021-12-31 21:00:00 CET,-5.6


In [24]:
m.write('unit_commitment_problem.lp')
m.write('unit_commitment_problem.mps')
m.Turb.solution.write_csv('turbine.csv')
m.Storage_level.solution.write_csv('storage.csv')
m.Pump.solution.write_csv('pump.csv')

Writing the model to unit_commitment_problem.lp
Writing the model to unit_commitment_problem.mps
